![Pytorch](../../../pytorch_logo_2018.svg)

# Pytorch 基础篇（3）：Logistic Regression（逻辑斯蒂回归）

**[参考代码](https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/logistic_regression/main.py)**

In [1]:
# 包
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# 超参数设置 Hyper-parameters
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## MINIST数据集加载（image and labels）

In [3]:
train_dataset = torchvision.datasets.MNIST(root='../../../data/minist', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

Processing...
Done!


In [5]:
test_dataset = torchvision.datasets.MNIST(root='../../../data/minist', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [6]:
# 数据加载器（data loader）
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Logistic Regression模型：加载和训练

In [7]:
# 线性模型，指定
model = nn.Linear(input_size, num_classes)

# 损失函数和优化器
# nn.CrossEntropyLoss()内部集成了softmax函数
# It is useful when training a classification problem with `C` classes.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 将图像序列抓换至大小为 (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播及优化
        optimizer.zero_grad() # 注意每次循环都要注意清空梯度缓存
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2091
Epoch [1/5], Step [200/600], Loss: 2.0910
Epoch [1/5], Step [300/600], Loss: 2.0584
Epoch [1/5], Step [400/600], Loss: 1.9679
Epoch [1/5], Step [500/600], Loss: 1.8440
Epoch [1/5], Step [600/600], Loss: 1.7939
Epoch [2/5], Step [100/600], Loss: 1.7501
Epoch [2/5], Step [200/600], Loss: 1.6417
Epoch [2/5], Step [300/600], Loss: 1.6071
Epoch [2/5], Step [400/600], Loss: 1.5562
Epoch [2/5], Step [500/600], Loss: 1.5750
Epoch [2/5], Step [600/600], Loss: 1.4774
Epoch [3/5], Step [100/600], Loss: 1.4367
Epoch [3/5], Step [200/600], Loss: 1.3702
Epoch [3/5], Step [300/600], Loss: 1.3308
Epoch [3/5], Step [400/600], Loss: 1.3523
Epoch [3/5], Step [500/600], Loss: 1.3248
Epoch [3/5], Step [600/600], Loss: 1.3202
Epoch [4/5], Step [100/600], Loss: 1.2332
Epoch [4/5], Step [200/600], Loss: 1.1691
Epoch [4/5], Step [300/600], Loss: 1.2277
Epoch [4/5], Step [400/600], Loss: 1.1631
Epoch [4/5], Step [500/600], Loss: 1.1385
Epoch [4/5], Step [600/600], Loss:

## 模型测试

In [8]:
# 在测试阶段，为了运行内存效率，就不需要计算梯度了
# Pytorch 默认每一次前向传播都会计算梯度
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82 %


In [9]:
## 保存模型
torch.save(model.state_dict(), 'model.ckpt')